In [13]:
import os
from task_utils import get_secret
api_key = get_secret()
os.environ["OPENAI_API_KEY"] = api_key

In [25]:
# format_guide_agent.py

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os

os.environ["OPENAI_API_KEY"] = api_key

model = ChatOpenAI(model="gpt-4o")
parser = StrOutputParser()

# Step 1 Agent: Guide user on formats & approaches
format_guide_prompt = ChatPromptTemplate.from_messages([
    ("user", 
"""
You are an AI guide that helps users decide how to build digital projects.

Given a human's idea, do the following:

1. Detect the final product format (choose from: website, document, poster, image, music, video, presentation).
2. Suggest 2–3 different implementation approaches for that format.
3. For each approach, explain what it is, its advantages, and its drawbacks in beginner-friendly language.
4. Ask the user to choose one approach.

DO NOT generate a template yet — stop after asking the user for their choice.

Here is the user’s idea:
{user_input}
""")
])

guide_chain = format_guide_prompt | model | parser

def run_format_guide(user_input):
    return guide_chain.invoke({"user_input": user_input})


In [34]:


model = ChatOpenAI(model="gpt-4o")
parser = StrOutputParser()

# Step 2 Agent: Build template based on user’s selected approach
template_builder_prompt = ChatPromptTemplate.from_messages([
    ("user", 
"""
You are an AI project assistant.

The user has selected this approach for their project:
Approach: {user_approach}

Their original goal was:
"{user_input}"

Based on this, do the following:

1. Reflect on how to make the output rich, comprehensive, and helpful to its audience.
2. Think of additional sections or features that could enhance it.
3. Then generate a JSON object with:

\{{
  "template": \{{
    "structure": ["list of relevant files or components"],
    "placeholders": ["sections that need to be filled in"]
  \}},
  "missing_data": ["specific information the user will need to gather"],
  "suggested_enhancements": ["optional bonus features or content ideas"]
\}}

4. Think of a more user-friendly summarization of the template, denoting section and subsetction by the number of # beforehand, such tahat
the user can easily see how the finla output is going to be.
e.g. if the user want to write a SOP for graduate program.
## Intro Paragraph
- a vivid introduction to how I came to love data science
- link to CMU core values
## Body Paragraph 1
= My research experience at Do.ai
- How it helped me grow
- How it links to my vision and CMU fit
## Body Paragraph 2
- ...

If the user wants a website, then dont' mention any jargon like php or html, just print the pages you want to create the directory this page can link to
and the content you want to display in this page, all using '-' beforehand.


5. For the template to be filled eith actual data later, queries need to be asked. Iterate thoguh each missing data, what google search needs to be 
performed; some may require user information beforehand, in that case specify the question and the search you will implement later. Summarize above as a sequential to-dolist
such that by following this to-do list searches can be complete and the template can be filled.
""")
])


template_chain = template_builder_prompt | model | parser

def run_template_builder(user_input, user_approach):
    return template_chain.invoke({
        "user_input": user_input,
        "user_approach": user_approach
    })


In [35]:
print("🔹 Welcome to the Human-AI Collaboration Platform!")
user_prompt = input("What would you like to build?\n> ")

print("\n🤖 Thinking about your options...\n")
guide_response = run_format_guide(user_prompt)
print(guide_response)

user_approach = input("\nWhich approach do you want to use?\n> ")

print("\n🔧 Generating a tailored template based on your choice...\n")
template_response = run_template_builder(user_prompt, user_approach)
print(template_response)

🔹 Welcome to the Human-AI Collaboration Platform!

🤖 Thinking about your options...

Great! Let's dive into your idea of creating a website for your personal blog.

### Final Product Format:
- **Website**

### Implementation Approaches:

1. **Using a Website Builder (e.g., WordPress, Wix, Squarespace):**

   - **What It Is:** These are platforms that offer pre-designed templates and drag-and-drop tools to build a website without needing to code.
   
   - **Advantages:**
     - User-friendly and quick to set up.
     - No coding skills required.
     - Offers many customizable templates and plugins.
     - Often includes hosting and other services bundled together.

   - **Drawbacks:**
     - Limited customization compared to coding from scratch.
     - Some features may be locked behind paid plans.
     - You might outgrow the platform if you want very specific functionalities in the future.

2. **Using a Content Management System (CMS) like WordPress.org:**

   - **What It Is:** This 